In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [17]:
data = pd.read_csv("../../HKDailyStocksQuotes/0001.HK.csv", index_col = 'Date')
data.index = pd.to_datetime(data.index,  format="%Y%m%d")

In [19]:
# data.index
# type(data.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [20]:
data

,RIC,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2013-12-09,0001.HK,88.625099,88.625099,87.338501,87.553001,72.656769,2352958
2013-12-10,0001.HK,87.910301,88.124702,87.410004,87.767403,72.834679,3761666
2013-12-11,0001.HK,87.410004,87.410004,85.766197,86.052101,71.411217,5240505
2013-12-12,0001.HK,85.408798,87.481499,85.408798,86.695297,71.944977,3750416
2013-12-13,0001.HK,85.694702,86.480904,85.408798,85.837601,71.233215,3781443
2013-12-16,0001.HK,86.266502,86.909698,85.766197,86.409401,71.707718,6103307
2013-12-17,0001.HK,85.766197,86.337997,84.336700,84.408203,70.047020,6440522
2013-12-18,0001.HK,85.480301,86.480904,85.051498,86.195000,71.529808,5331563
2013-12-19,0001.HK,87.838898,87.838898,84.551102,85.194397,70.699440,5606428


In [68]:
bins = [-np.inf, -0.1, -0.05, -0.03, 0, 0.03, 0.05, 0.1, np.inf]
names = ['<-0.1', '-0.1--0.05', '-0.05--0.03', ' -0.03-0', '0-0.03', '0.03-0.05', '0.05-0.1', '>0.1']

def turning_points(array):
    """ turning_points(array) -> min_indices, max_indices
    Finds the turning points within an 1D array and returns the indices of the minimum and
    maximum turning points in two separate lists.
    """
    idx_max, idx_min = [], []
    if len(array) < 3:
        return idx_min, idx_max

    NEUTRAL, RISING, FALLING = range(3)

    def get_state(a, b):
        if a < b: return RISING
        if a > b: return FALLING
        return NEUTRAL

    ps = get_state(array[0], array[1])
    begin = 1
    for i in range(2, len(array)):
        s = get_state(array[i - 1], array[i])
        if s != NEUTRAL:
            if ps != NEUTRAL and ps != s:
                if s == FALLING:
                    idx_max.append((begin + i - 1) // 2)
                else:
                    idx_min.append((begin + i - 1) // 2)
            begin = i
            ps = s
    return idx_min, idx_max 

def create_turning_point_matrix_for_day_diff(data, day_diff, id_tp_array):
    # shift the turning point array to certain day difference, with the close price of turning point 
    id_tp_array = pd.DataFrame({'max' : [int(i in id_tp_array) for i in range(len(data))]})
    id_tp_array = id_tp_array.set_index(data.index)
    idx_tp_shift = id_tp_array.shift(day_diff)
    px_shift = data['Adj Close'].shift(day_diff)
    # cal px diff of Tday's px and TP's px
    px_diff = ((data['Adj Close'] - px_shift) / px_shift).mul(idx_tp_shift['max'], fill_value=0)
    
    
    # binning and transform to one hot categorization
    # fibonacci_bins = [-np.inf, -0.764, -0.618, -0.5, -0.382, 0, 0.382, 0.5, 0.618, 0.764, np.inf]
    
    px_diff_bin = pd.cut(px_diff, bins, labels=names)
    px_diff_bin = pd.get_dummies(px_diff_bin)
    px_diff_bin[px_diff==0] = 0 # px-diff==0 is matched into 1 bin, but it should be zero in all cols

    return px_diff_bin

def create_turning_point_3d_matrix(data):
    if data is None:
        return None, None
    # create turning points series
    idx_min, idx_max = turning_points(data['Adj Close'])

    max_matrix2 = create_turning_point_matrix_for_day_diff(data, 2, idx_max)
    max_matrix3 = create_turning_point_matrix_for_day_diff(data, 3, idx_max)
    max_matrix5 = create_turning_point_matrix_for_day_diff(data, 5, idx_max)
    max_matrix10 = create_turning_point_matrix_for_day_diff(data, 10, idx_max)
    max_matrix15 = create_turning_point_matrix_for_day_diff(data, 15, idx_max)

    result_max = pd.concat([max_matrix2, max_matrix3, max_matrix5, max_matrix10, max_matrix15], keys=[2, 3, 5, 10, 15])
    result_max.index = result_max.index.swaplevel(1, 0)
    result_max = result_max.sort_index()
    result_max.index.names = ['Date', 'DayDiff']

    min_matrix2 = create_turning_point_matrix_for_day_diff(data, 2, idx_min)
    min_matrix3 = create_turning_point_matrix_for_day_diff(data, 3, idx_min)
    min_matrix5 = create_turning_point_matrix_for_day_diff(data, 5, idx_min)
    min_matrix10 = create_turning_point_matrix_for_day_diff(data, 10, idx_min)
    min_matrix15 = create_turning_point_matrix_for_day_diff(data, 15, idx_min)

    result_min = pd.concat([min_matrix2, min_matrix3, min_matrix5, min_matrix10, min_matrix15], keys=[2, 3, 5, 10, 15])
    # rotate the matrix axes
    result_min.index = result_min.index.swaplevel(1, 0)
    result_min = result_min.sort_index()
    result_min.index.names = ['Date', 'DayDiff']

    return result_max, result_min

def create_technical_indicator_3d_matrix(data):
    if data is None:
        return None
    high_low_diff = (data['High'] - data['Low']) / data['Low']
    ma5_diff = (sma(data['Close'], 5) - data['Close']) / data['Close']
    ma10_diff = (sma(data['Close'], 10) - data['Close']) / data['Close']
    ma20_diff = (sma(data['Close'], 20) - data['Close']) / data['Close']

    high_low_diff_bin = pd.cut(high_low_diff, bins, labels=names)
    ma5_diff_bin = pd.cut(ma5_diff, bins, labels=names)
    ma10_diff_bin = pd.cut(ma10_diff, bins, labels=names)
    ma20_diff_bin = pd.cut(ma20_diff, bins, labels=names)
    
    result_hl_diff = pd.get_dummies(high_low_diff_bin)
    result_ma5_diff = pd.get_dummies(ma5_diff_bin)
    result_ma10_diff = pd.get_dummies(ma10_diff_bin)
    result_ma20_diff = pd.get_dummies(ma20_diff_bin)
    
    result_matrix = pd.concat([result_hl_diff, result_ma5_diff, result_ma10_diff, result_ma20_diff], keys=['hl_diff', 'ma5_diff', 'ma10_diff', 'ma20_diff'])
    # rotate result matrix axes
    result_matrix.index = result_matrix.index.swaplevel(1, 0)
    result_matrix = result_matrix.sort_index()
    result_matrix.index.names = ['Date', 'Indicator']
    
    return result_matrix

def sma(data, period=5):
    return data.rolling(period).mean()

In [242]:
# idx_min, idx_max = turning_points(data['Adj Close'])


# plt.plot(data[:100].index, pd.Series(max_array)[:100]*30)
# plt.plot(data['Adj Close'][:100], marker='o', markevery=idx_min)
# plt.plot(data['Adj Close'][:100], marker='o', markevery=idx_max)
# plt.ylabel('some numbers')
# plt.show()

In [44]:
result_max, result_min = create_turning_point_3d_matrix(data)

In [67]:
# result_max.groupby('DayDiff').sum()
# result_max
create_technical_indicator_3d_matrix(data)

<-0.1  -0.1--0.05  -0.05--0.03   -0.03-0  0-0.03  \
Date                                                                     
2013-12-09 hl_diff        0           0            0         0       1   
           ma10_diff      0           0            0         0       0   
           ma20_diff      0           0            0         0       0   
           ma5_diff       0           0            0         0       0   
2013-12-10 hl_diff        0           0            0         0       1   
           ma10_diff      0           0            0         0       0   
           ma20_diff      0           0            0         0       0   
           ma5_diff       0           0            0         0       0   
2013-12-11 hl_diff        0           0            0         0       1   
           ma10_diff      0           0            0         0       0   
           ma20_diff      0           0            0         0       0   
           ma5_diff       0           0            0         0       0   
2013-12-12 hl_diff        0           0            0         0       1   
           ma10_diff      0           0            0         0       0   
           ma20_diff      0           0            0         0       0   
           ma5_diff       0           0            0         0       0   
2013-12-13 hl_diff        0           0            0         0       1   
           ma10_diff      0           0            0         0       0   
           ma20_diff      0           0            0         0       0   
           ma5_diff       0           0            0         0       1   
2013-12-16 hl_diff        0           0            0         0       1   
           ma10_diff      0           0            0         0       0   
           ma20_diff      0           0            0         0       0   
           ma5_diff       0           0            0         0       1   
2013-12-17 hl_diff        0           0            0         0       1   
           ma10_diff      0           0            0         0       0   
           ma20_diff      0           0            0         0       0   
           ma5_diff       0           0            0         0       1   
2013-12-18 hl_diff        0           0            0         0       1   
           ma10_diff      0           0            0         0       0   
...                     ...         ...          ...       ...     ...   
2018-11-28 ma20_diff      0           0            0         1       0   
           ma5_diff       0           0            0         1       0   
2018-11-29 hl_diff        0           0            0         0       1   
           ma10_diff      0           0            0         0       1   
           ma20_diff      0           0            0         1       0   
           ma5_diff       0           0            0         0       1   
2018-11-30 hl_diff        0           0            0         0       1   
           ma10_diff      0           0            0         0       1   
           ma20_diff      0           0            0         1       0   
           ma5_diff       0           0            0         0       1   
2018-12-03 hl_diff        0           0            0         0       1   
           ma10_diff      0           0            0         1       0   
           ma20_diff      0           0            0         1       0   
           ma5_diff       0           0            0         1       0   
2018-12-04 hl_diff        0           0            0         0       1   
           ma10_diff      0           0            0         1       0   
           ma20_diff      0           0            0         1       0   
           ma5_diff       0           0            0         1       0   
2018-12-05 hl_diff        0           0            0         0       1   
           ma10_diff      0           0            0         0       1   
           ma20_diff      0           0            0         0       1   
           ma5_diff       